# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from dotenv import load_dotenv
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
google_api_key = os.getenv("google_api_key")

# Access maps with unique API key
gmaps.configure(api_key=google_api_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load cities to a dataframe
cities = pd.read_csv('output_data/cities.csv')
cities_df = pd.DataFrame(cities)
cities_df.head()



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Busselton,62,AU,1630120123,63,-33.6500,115.3333,61.05,9.55
1,Cape Town,72,ZA,1630119942,93,-33.9258,18.4232,44.89,4.00
2,Nikolskoye,61,RU,1630120123,93,59.7035,30.7861,57.09,5.55
3,Chui,1,UY,1630120124,81,-33.6971,-53.4616,47.55,5.55
4,Hilo,90,US,1630120124,75,19.7297,-155.0900,82.27,5.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
locations = cities_df[['Lat','Lng']].astype(float)
humidity = cities_df['Humidity'].astype(float)
# humidity heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,weights=humidity,
                                dissipating = False, max_intensity = max(cities_df['Humidity']),
                                point_radius =2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = cities_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
29,Saryagash,0,KZ,1630120131,16,41.4511,69.1701,78.76,4.68
50,Marsaxlokk,0,MT,1630120137,83,35.8419,14.5431,79.09,6.91
53,Gold Coast,0,AU,1630120050,34,-28.0000,153.4333,78.24,3.00
104,Yingkou,0,CN,1630120153,47,40.6648,122.2283,76.98,8.25
119,Bodrum,0,TR,1630120158,82,37.0383,27.4292,76.48,4.32
201,Caravelas,0,BR,1630120182,85,-17.7125,-39.2481,70.21,5.28
223,Lipari,0,IT,1630120189,61,38.4674,14.9540,76.95,7.94
240,Santa Cruz de Tenerife,0,ES,1630120194,63,28.4682,-16.2546,74.80,1.99
255,Progreso,0,MX,1630120198,87,21.2833,-89.6667,76.10,6.71
299,Mountain Home,0,US,1630120218,28,43.1330,-115.6912,72.34,9.73


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": google_api_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 29: Saryagash.
Closest hotel is fatyCentre Paradise.
------------
Retrieving Results for Index 50: Marsaxlokk.
Closest hotel is Water's Edge.
------------
Retrieving Results for Index 53: Gold Coast.
Closest hotel is Mantra Towers of Chevron Surfers Paradise.
------------
Retrieving Results for Index 104: Yingkou.
Closest hotel is Yingkou Juyin Junyue Hotel.
------------
Retrieving Results for Index 119: Bodrum.
Closest hotel is The Marmara Bodrum.
------------
Retrieving Results for Index 201: Caravelas.
Closest hotel is Pousada dos Navegantes.
------------
Retrieving Results for Index 223: Lipari.
Closest hotel is Hotel Villa Diana Soc. Clelia Hunziker s.a.s..
------------
Retrieving Results for Index 240: Santa Cruz de Tenerife.
Closest hotel is Hotel Silken Atlántida Santa Cruz.
------------
Retrieving Results for Index 255: Progreso.
Closest hotel is Playa Linda Hotel.
------------
Retrieving Results for Index 299: Mountain Home.
Closest hotel is Mount

In [24]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
29,Saryagash,0,KZ,1630120131,16,41.4511,69.1701,78.76,4.68,fatyCentre Paradise
50,Marsaxlokk,0,MT,1630120137,83,35.8419,14.5431,79.09,6.91,Water's Edge
53,Gold Coast,0,AU,1630120050,34,-28.0000,153.4333,78.24,3.00,Mantra Towers of Chevron Surfers Paradise
104,Yingkou,0,CN,1630120153,47,40.6648,122.2283,76.98,8.25,Yingkou Juyin Junyue Hotel
119,Bodrum,0,TR,1630120158,82,37.0383,27.4292,76.48,4.32,The Marmara Bodrum
201,Caravelas,0,BR,1630120182,85,-17.7125,-39.2481,70.21,5.28,Pousada dos Navegantes
223,Lipari,0,IT,1630120189,61,38.4674,14.9540,76.95,7.94,Hotel Villa Diana Soc. Clelia Hunziker s.a.s.
240,Santa Cruz de Tenerife,0,ES,1630120194,63,28.4682,-16.2546,74.80,1.99,Hotel Silken Atlántida Santa Cruz
255,Progreso,0,MX,1630120198,87,21.2833,-89.6667,76.10,6.71,Playa Linda Hotel
299,Mountain Home,0,US,1630120218,28,43.1330,-115.6912,72.34,9.73,Mountain Home Inn


In [26]:
# NOTE: Do not change any of the code in this cell

# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [27]:
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))